In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
class BoilerDataSet(object):
    """
    first run data_preparation.py to generate data.csv
    prepare boiler training and validation dataset
    simple version(small action dimension)

    """
    
    def __init__(self, num_steps, val_ratio=0.1):
        self.num_steps = num_steps  # time steps
        self.val_ratio = val_ratio  # train test ratio
        
        # Read csv file
        self.raw_data = pd.read_csv("./data/sim_train.csv", index_col='时间戳')

        # sort csv file
        cols = self.raw_data.columns.tolist()
        # print("origin len: {0}".format(len(cols)))
        cols = (cols[51:52] + cols[53:59] + cols [60:61] + cols[62:63] + cols[150:152]   # external input 
            + cols[0:50] + cols[52:53] + cols[122:139]  # Coal Pulverizing state
            + cols[50:51] + cols[59:60] + cols[61:62] + cols[63:101] + cols[112:114] + cols[118:122] + cols[139:145] + cols[146:149] + cols[152:158]    # Burning state
            + cols[101:112] + cols[114:118] + cols[145:146] + cols[149:150] # Steam Circulation state
            + cols[158:173] + cols[196:202] # Coal Pulverizing action
            + cols[173:192]                 # Burning action
            + cols[192:196])                # Steam Circulation action
        # print("ordered len: {0}".format(len(cols)))
        self.raw_data = self.raw_data[cols]

        # divide train set and valid set
        self.train_X, self.train_y, self.valid_X, self.valid_y = self.prepare_data(self.raw_data)

    def prepare_data(self, data):
        # split into groups of num_steps

        # 取出输入数据，学习num_steps步长的历史，iloc：通过行号获取行数据
        X = np.array([data.iloc[i: i + self.num_steps].values
                    for i in range(len(data) - self.num_steps)])

        # 取出输出数据，预测第num_steps步的值训练，ix / loc 可以通过行号和行标签进行索引
        # 这里只要对状态量进行预测即可，0-157列为 'A磨煤机电流':'大渣可燃物含量'
        y = np.array([data.iloc[i + self.num_steps, 0:158].values
                    for i in range(len(data) - self.num_steps)])

        train_size = int(len(X) * (1.0 - self.val_ratio))
        train_X, valid_X = X[:train_size], X[train_size:]
        train_y, valid_y = y[:train_size], y[train_size:]
        return train_X, train_y, valid_X, valid_y

    def generate_one_epoch(self, data_X, data_y, batch_size):
        num_batches = int(len(data_X)) // batch_size
        # if batch_size * num_batches < len(self.train_X):
        #     num_batches += 1

        batch_indices = list(range(num_batches))
        random.shuffle(batch_indices)
        for j in batch_indices:
            batch_X = data_X[j * batch_size: (j + 1) * batch_size]
            batch_y = data_y[j * batch_size: (j + 1) * batch_size]
            yield batch_X, batch_y

In [3]:
# class sim_config(object):
num_steps = 10
valid_ratio = 0.2

input_size = 202
num_neurons = 160
num_layers = 3
output_size = 158

learning_rate = 0.001
learning_rate_decay = 0.95

max_epoch = 50
batch_size = 1

save_log_iter = 10
display_iter = 20

tf.random.set_seed(2022)

In [4]:
class SimulatorRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.rnn_cell = tf.keras.layers.LSTMCell(units)
                        # tf.keras.layers.SimpleRNNCell(units, # LSTMCell(units,
                        #                                activation=None)
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.activation = tf.keras.activations.get(activation)

    def call(self, inputs, states):
        outputs, new_states = self.rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [5]:
class SimulatorRNNLayer(tf.keras.layers.Layer):
    def __init__(self, cell, return_sequences=False, **kwargs):
        super().__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences

    def get_initial_state(self, inputs):
        try:
            return self.cell.get_initial_state(inputs)
        except AttributeError:
            # fallback to zeros if self.cell has no get_initial_state() method
            batch_size = tf.shape(inputs)[0]
            return [tf.zeros([batch_size, self.cell.state_size],
                             dtype=inputs.dtype)]

    @tf.function
    def call(self, inputs):
        states = self.get_initial_state(inputs)
        shape = tf.shape(inputs)
        batch_size = shape[0]
        n_steps = shape[1]
        sequences = tf.TensorArray(
            inputs.dtype, size=(n_steps if self.return_sequences else 0))
        outputs = tf.zeros(shape=[batch_size, self.cell.output_size],
                           dtype=inputs.dtype)
        for step in tf.range(n_steps):
            outputs, states = self.cell(inputs[:, step], states)
            if self.return_sequences:
                sequences = sequences.write(step, outputs)

        if self.return_sequences:
            # stack the outputs into an array of shape
            # [time steps, batch size, dims], then transpose it to shape
            # [batch size, time steps, dims]
            return tf.transpose(sequences.stack(), [1, 0, 2])
        else:
            return outputs

In [6]:
def SimulatorRNNModel():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=[num_steps, input_size], name="inputs"),
        tf.keras.layers.LSTM(256, return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.RNN(SimulatorRNNCell(128), input_shape=[None, num_steps, input_size]),
        tf.keras.layers.Dense(158, name="targets")
    ])

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(), 
        optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate), 
        metrics=['mean_absolute_error']
    )

    return model

model = SimulatorRNNModel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 256)           470016    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 targets (Dense)             (None, 158)               20382     
                                                                 
Total params: 687,518
Trainable params: 687,518
Non-trainable params: 0
_________________________________________________________________


In [15]:
def fit_and_evaluate(model, train_X, train_y, valid_X, valid_y, learning_rate, batch_size=1, epochs=500):
    callback_list = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", 
            patience=50, 
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath='./logs/LSTM/saved_models/model-{epoch:02d}-{val_loss:.4f}.h5',
            monitor="val_loss",
            verbose=1,
            save_weights_only=True,
            save_best_only=True,
        ),
        tf.keras.callbacks.TensorBoard(log_dir='./logs/')
    ]   
    
    history = model.fit(
        x=train_X, y=train_y,
        epochs=epochs, 
        validation_data=(valid_X, valid_y),
        callbacks=callback_list)
    valid_loss, valid_mae = model.evaluate(x=valid_X, y=valid_y) # Returns the loss value & metrics values for the model in test mode
    return valid_mae * 1e6  # valid mean absolute error

In [16]:
# read data
boiler_dataset = BoilerDataSet(num_steps=num_steps, val_ratio=valid_ratio)
train_X, train_y = boiler_dataset.train_X, boiler_dataset.train_y
valid_X, valid_y = boiler_dataset.valid_X, boiler_dataset.valid_y

fit_and_evaluate(model, train_X, train_y, valid_X, valid_y, learning_rate, batch_size, max_epoch)

Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 7.7237e-04 - mean_absolute_error: 0.0210
Epoch 1: val_loss improved from inf to 0.00219, saving model to ./logs/LSTM/saved_models\model-01-0.0022.h5
1/1 [==============================] - 0s 220ms/step - loss: 7.7237e-04 - mean_absolute_error: 0.0210 - val_loss: 0.0022 - val_mean_absolute_error: 0.0271
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 7.1251e-04 - mean_absolute_error: 0.0201
Epoch 2: val_loss did not improve from 0.00219
1/1 [==============================] - 0s 96ms/step - loss: 7.1251e-04 - mean_absolute_error: 0.0201 - val_loss: 0.0025 - val_mean_absolute_error: 0.0299
Epoch 3/50
1/1 [==============================] - ETA: 0s - loss: 6.8528e-04 - mean_absolute_error: 0.0197
Epoch 3: val_loss improved from 0.00219 to 0.00213, saving model to ./logs/LSTM/saved_models\model-03-0.0021.h5
1/1 [==============================] - 0s 113ms/step - loss: 6.8528e-04 - mean_absolute_error: 0.0197 -

25075.042620301247